In [ ]:
!pip install openai==0.28

In [ ]:
import pandas as pd
import openai
import time
import os
import re

# ========== 사전 설정 ==========
openai.api_key = ''
MAX_RETRIES = 3
PROMPT_MAX_LENGTH = 2000
MODEL = 'gpt-4o-mini'
SYSTEM_MESSAGE = "당신은 한국어로 답변하는 지능형 투자 분석가입니다."
TARGET_COMPANY = "삼성전자"

# ========== 프롬프트 생성 함수 ==========
def generate_risk_seeking_prompt(content):
    return f"""
당신은 높은 수익을 추구하며, 리스크를 감수하는 투자 전문가입니다.
분석 대상 기업: {TARGET_COMPANY}

다음 뉴스 내용을 바탕으로 {TARGET_COMPANY}의 주가 예측을 해주세요.
가능한 예측: 상승, 하락, 횡보, 판단불가
또한, 예측에 대한 확신도를 0과 1 사이의 숫자로 제시하고,
이 뉴스 이벤트가 주가 움직임에 얼마나 영향력이 있다고 생각하는지,
그 근거를 제시하고, 이를 0~1 사이 중요도로 표현해주세요.

[뉴스 내용]
{content}

[답변 형식 예시]
예측: 상승
확신도:0.8
근거: 이 뉴스는 기업 핵심 제품의 판매 증가를 시사하며 실적 개선 가능성이 큼
중요도:0.7
""".strip()

def generate_risk_averse_prompt(content):
    return f"""
당신은 안정적인 수익과 자본 보전을 중시하는 투자 전문가입니다.
분석 대상 기업: {TARGET_COMPANY}

다음 뉴스 내용을 바탕으로 {TARGET_COMPANY}의 주가 예측을 해주세요.
가능한 예측: 상승, 하락, 횡보, 판단불가
또한, 예측에 대한 확신도를 0과 1 사이의 숫자로 제시하고,
이 뉴스 이벤트가 주가 움직임에 얼마나 영향력이 있다고 생각하는지,
그 근거를 제시하고, 이를 0~1 사이 중요도로 표현해주세요.

[뉴스 내용]
{content}

[답변 형식 예시]
예측: 하락
확신도:0.6
근거: 이 뉴스는 원자재 가격 상승으로 인한 마진 축소 가능성을 시사
중요도:0.5
""".strip()

def generate_mediator_prompt(risk_seeking_prediction, risk_seeking_conf, risk_seeking_reason, risk_seeking_importance,
                             risk_averse_prediction, risk_averse_conf, risk_averse_reason, risk_averse_importance):
    return f"""
당신은 중립적인 투자 분석가입니다.
분석 대상 기업: {TARGET_COMPANY}

아래 두 전문가(위험 추구형, 위험 회피형)의 예측 결과, 확신도, 근거, 중요도를 참고하여
반드시 {TARGET_COMPANY} 주가 예측을 상승, 하락, 횡보 중 하나로 선택하세요.
또한 예측에 대한 확신도를 0과 1 사이 숫자로 제시하고,
이 뉴스 이벤트가 주가 움직임에 얼마나 영향력이 있다고 생각하는지, 그 근거를 제시하고 0~1 사이 중요도로 표현해주세요.

[위험 추구형 전문가]
예측: {risk_seeking_prediction}
확신도:{risk_seeking_conf}
근거:{risk_seeking_reason}
중요도:{risk_seeking_importance}

[위험 회피형 전문가]
예측: {risk_averse_prediction}
확신도:{risk_averse_conf}
근거:{risk_averse_reason}
중요도:{risk_averse_importance}

[답변 형식 예시]
예측: 상승
확신도:0.7
근거: 두 전문가의 의견 중 상승 가능성을 시사하는 부분이 더 설득력 있음
중요도:0.6
""".strip()

# ========== API 응답 함수 ==========
def get_agent_response(prompt):
    for attempt in range(MAX_RETRIES):
        try:
            response = openai.ChatCompletion.create(
                model=MODEL,
                messages=[
                    {'role': 'system', 'content': SYSTEM_MESSAGE},
                    {'role': 'user', 'content': prompt}
                ],
                max_tokens=500,
                n=1,
                temperature=0.7,
            )
            content = response.choices[0].message.content.strip()
            return content if content else ""
        except Exception as e:
            print(f"오류 발생: {e}")
            if attempt < MAX_RETRIES - 1:
                print("잠시 후 재시도합니다...")
                time.sleep(5)
            else:
                print("최대 재시도 횟수 초과. 빈 응답 반환")
                return ""

# ========== 응답 파싱 함수 ==========
def parse_response(response):
    pred_pattern = r"예측:\s*(상승|하락|횡보|판단불가)"
    conf_pattern = r"확신도:\s*([0-1](?:\.\d+)?)"
    imp_pattern = r"중요도:\s*([0-1](?:\.\d+)?)"
    reason_pattern = r"근거:\s*(.*)"

    prediction_match = re.search(pred_pattern, response)
    confidence_match = re.search(conf_pattern, response)
    importance_match = re.search(imp_pattern, response)
    reason_match = re.search(reason_pattern, response)

    prediction = prediction_match.group(1) if prediction_match else "판단불가"
    confidence = float(confidence_match.group(1)) if confidence_match else 0.0
    importance = float(importance_match.group(1)) if importance_match else 0.0
    reason = reason_match.group(1).strip() if reason_match else "근거 없음"

    return prediction, confidence, reason, importance

# ========== 메인 로직 ==========
def process_articles(df):
    rs_preds = []
    rs_confs = []
    rs_reasons = []
    rs_imps = []

    ra_preds = []
    ra_confs = []
    ra_reasons = []
    ra_imps = []

    mediator_preds = []
    mediator_confs = []
    mediator_reasons = []
    mediator_imps = []

    total = len(df)

    for i, (idx, row) in enumerate(df.iterrows(), start=1):
        content = row['내용'][:PROMPT_MAX_LENGTH]

        # 위험 추구형
        rs_prompt = generate_risk_seeking_prompt(content)
        rs_response = get_agent_response(rs_prompt)
        rs_pred, rs_conf, rs_reason, rs_imp = parse_response(rs_response)

        # 위험 회피형
        ra_prompt = generate_risk_averse_prompt(content)
        ra_response = get_agent_response(ra_prompt)
        ra_pred, ra_conf, ra_reason, ra_imp = parse_response(ra_response)

        # 중재자
        mediator_prompt = generate_mediator_prompt(rs_pred, rs_conf, rs_reason, rs_imp,
                                                   ra_pred, ra_conf, ra_reason, ra_imp)
        mediator_response = get_agent_response(mediator_prompt)
        m_pred, m_conf, m_reason, m_imp = parse_response(mediator_response)

        # 중재자 강제 조건 (판단불가 금지)
        if m_pred == "판단불가":
            m_pred = "횡보"
            if m_conf == 0.0:
                m_conf = 0.5
            if m_imp == 0.0:
                m_imp = 0.5
            if m_reason == "근거 없음":
                m_reason = "중재 결과 판단불가를 회피하기 위한 기본 근거"

        rs_preds.append(rs_pred)
        rs_confs.append(rs_conf)
        rs_reasons.append(rs_reason)
        rs_imps.append(rs_imp)

        ra_preds.append(ra_pred)
        ra_confs.append(ra_conf)
        ra_reasons.append(ra_reason)
        ra_imps.append(ra_imp)

        mediator_preds.append(m_pred)
        mediator_confs.append(m_conf)
        mediator_reasons.append(m_reason)
        mediator_imps.append(m_imp)

        # 진행 상황 및 응답 출력
        print(f"\n[{i}/{total} 개 기사 처리 완료]")
        print("=== 위험 추구형 응답 ===")
        print(rs_response)
        print("=== 위험 회피형 응답 ===")
        print(ra_response)
        print("=== 중재자 응답 ===")
        print(mediator_response)

    # 결과 저장
    df['위험추구형_예측'] = rs_preds
    df['위험추구형_확신도'] = rs_confs
    df['위험추구형_근거'] = rs_reasons
    df['위험추구형_중요도'] = rs_imps

    df['안전추구형_예측'] = ra_preds
    df['안전추구형_확신도'] = ra_confs
    df['안전추구형_근거'] = ra_reasons
    df['안전추구형_중요도'] = ra_imps

    df['중재자_예측'] = mediator_preds
    df['중재자_확신도'] = mediator_confs
    df['중재자_근거'] = mediator_reasons
    df['중재자_중요도'] = mediator_imps

    return df

# 중요도와 확신도를 활용한 가중치 기반 최종 결정
def finalize_decision_with_weights(df):
    # 예측 결과 별로 가중점수 합산
    scores = {'상승': 0.0, '하락': 0.0, '횡보': 0.0}

    for idx, row in df.iterrows():
        pred = row['중재자_예측']
        conf = row['중재자_확신도']
        imp = row['중재자_중요도']

        weighted_score = conf * imp
        if pred in scores:
            scores[pred] += weighted_score
        else:
            scores[pred] = weighted_score

    final_decision = max(scores, key=scores.get)  # 가장 높은 점수를 가진 예측 방향
    print("\n중재 에이전트의 가중치 기반 예측 점수:")
    print(scores)
    print(f"\n가중치 기반 최종 결정된 상태: {final_decision}")
    return final_decision

# ========== 실행 예시 ==========
if __name__ == "__main__":
    df = pd.read_csv('/naver_news_articles.csv')
    
    df_processed = process_articles(df)

    # 중요도*확신도 가중치 방식으로 최종 결정
    final_decision = finalize_decision_with_weights(df_processed)

    # 처리 결과 일부 확인
    print(df_processed[['내용',
                        '위험추구형_예측','위험추구형_확신도','위험추구형_근거','위험추구형_중요도',
                        '안전추구형_예측','안전추구형_확신도','안전추구형_근거','안전추구형_중요도',
                        '중재자_예측','중재자_확신도','중재자_근거','중재자_중요도']].head())

In [ ]:
df_processed.to_csv('/agent_predictions_sampled.csv',
                    index=False,
                    encoding='utf-8-sig')